In [ ]:
!pip install langchain faiss-cpu pdfplumber langchain-google-genai tiktoken
!pip install -U langchain-community

import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google.colab import files
uploaded = files.upload()

import pdfplumber

pdf_text = ""
for fn in uploaded:
    with pdfplumber.open(fn) as pdf:
        for page in pdf.pages:
            pdf_text += page.extract_text() + "\n"


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents([pdf_text])

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(documents, embeddings)

from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

while True:
    question = input("Ask something about the PDF (type 'exit' to quit): ")
    if question.lower() in ['exit', 'quit']:
        break
    result = qa_chain.run(question)
    print("Answer:", result)
